In [10]:
import requests

# 调用Binance API获取比特币/美元交易对的市场深度
url = "https://api.binance.com/api/v3/depth?symbol=BTCUSDT&limit=0"
response = requests.get(url)

# 解析API响应并提取所需信息
data = response.json()

bids = data['bids'][0]  # 最高买价和买量
asks = data['asks'][0]  # 最低卖价和卖量

print("最高买价: ", bids[0])
print("最高买量: ", bids[1])
print("最低卖价: ", asks[0])
print("最低卖量: ", asks[1])


KeyError: 'bids'

In [4]:
import ccxt
import pandas as pd
import numpy as np

def moving_average(data, window):
    return np.convolve(data, np.ones(window), 'valid') / window

symbol = 'BTC/USDT'
timeframe = '1h'
stop_loss_rate = 1
take_profit_rate = 0.2

binance = ccxt.binance()
ohlcv = binance.fetch_ohlcv(symbol, timeframe)
ohlcv_df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

ohlcv_df['ma5'] = ohlcv_df['close'].rolling(window=5).mean()
ohlcv_df['ma8'] = ohlcv_df['close'].rolling(window=8).mean()
ohlcv_df['ma13'] = ohlcv_df['close'].rolling(window=13).mean()

in_position = False
buy_price = 0
winning_trades = 0
total_trades = 0

for index, row in ohlcv_df.iterrows():
    if not in_position and row['ma5'] > row['ma8'] and row['ma5'] > row['ma13']:
        in_position = True
        buy_price = row['close']
    elif in_position:
        stop_loss = buy_price * (1 - stop_loss_rate)
        take_profit = buy_price * (1 + take_profit_rate)
        if row['low'] <= stop_loss:
            in_position = False
            total_trades += 1
        elif row['high'] >= take_profit:
            in_position = False
            total_trades += 1
            winning_trades += 1

win_rate = winning_trades / total_trades
print(f"Winning trades: {winning_trades}")
print(f"Total trades: {total_trades}")
print(f"Win rate: {win_rate * 100:.2f}%")


ZeroDivisionError: division by zero